<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Michael/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = Images.astype(float)

In [58]:
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float, max_rows = 1000)
controlData[:,0] /= np.max(controlData[:,0])

15.0
(1000, 2)
(1000, 2)


In [129]:
from torch.utils.data import TensorDataset
ImagesTrain = torch.tensor(Images[0:799])
steerDataTrain = torch.tensor(controlData[0:799, 0])
steerDataTrain = steerDataTrain.unsqueeze(1)
trainSteerDataset = TensorDataset(ImagesTrain, steerDataTrain)
ImagesTest = torch.tensor(Images[800:999])
steerDataTest = torch.tensor(controlData[800:999, 0])
steerDataTest = steerDataTest.unsqueeze(1)
testSteerDataset = TensorDataset(ImagesTest, steerDataTest)
throttleDataTrain = torch.tensor(controlData[0:799, 1])
throttleDataTrain = throttleDataTrain.unsqueeze(1)
trainThrottleDataset = TensorDataset(ImagesTrain, throttleDataTrain)
throttleDataTest = torch.tensor(controlData[800:999, 1])
throttleDataTest = throttleDataTest.unsqueeze(1)
testThrottleDataset = TensorDataset(ImagesTest, throttleDataTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [125]:
steerTrainDataLoader = torch.utils.data.DataLoader(trainSteerDataset, batch_size=32, shuffle=True, drop_last=True)
steerTestDataLoader = torch.utils.data.DataLoader(testSteerDataset, batch_size=32, shuffle=False, drop_last=True)
throttleTrainDataLoader = torch.utils.data.DataLoader(trainThrottleDataset, batch_size=16, shuffle=True, drop_last=True)
throttleTestDataLoader = torch.utils.data.DataLoader(testThrottleDataset, batch_size=16, drop_last=True, shuffle=False)

In [126]:
class LR(torch.nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=5, kernel_size=(5, 5))
        self.relu1 = torch.nn.ReLU()
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = torch.nn.Conv2d(in_channels=5, out_channels=10, kernel_size=(5, 5))
        self.relu2 = torch.nn.ReLU()
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.linear1 = torch.nn.Linear(28490, 10)
        self.linear2 = torch.nn.Linear(10, 1)
        self.relu3 = torch.nn.ReLU()
    
    def forward(self, x):
      x = torch.unsqueeze(x, 1)
      x = self.conv1(x)
      x = self.relu1(x)
      x = self.maxpool1(x)
      x = self.conv2(x)
      #x = self.relu2(x)
      x = self.maxpool2(x)
      x = torch.flatten(x, 1)
      x = self.linear1(x)
      #x = self.relu3(x)
      x = self.linear2(x)
      output = x
      return output

steerNet = LR()
steerOptimizer = torch.optim.Adam(steerNet.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()
criterionThrottle = torch.nn.MSELoss()
throttleNet = LR()
throttleOptimizer = torch.optim.Adam(throttleNet.parameters(), lr=.5)

In [131]:
#steering trainer
train_loss_history = []
test_loss_history = []
for epoch in range(3):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(steerTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        steerOptimizer.zero_grad()
        #print(np.shape(images))
        predicted_output = steerNet(images.float())
        #print(predicted_output)
        #print("HERE" + str(predicted_output))
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        steerOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(steerTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = steerNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(steerTrainDataLoader)
    test_loss = test_loss/len(steerTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

Epoch 0 finished with train loss 520.5802091695368 and test loss 27.02639897664388
Epoch 1 finished with train loss 17.779947598775227 and test loss 3.4727513790130615
Epoch 2 finished with train loss 1.8384169712662697 and test loss 0.6980660408735275


In [130]:
#throttle trainer
train_loss_history = []
test_loss_history = []
for epoch in range(10):
    train_loss = 0.0
    test_loss = 0.0
    for i, data in enumerate(throttleTrainDataLoader):
        images, labels = data
        images = images.to(torch.float32)
        labels = labels.to(torch.float32)
        throttleOptimizer.zero_grad()
        predicted_output = throttleNet(images.float())
        #print(predicted_output)
        predicted_output = predicted_output.to(torch.float32)
        loss = criterion(predicted_output, labels)
        loss.backward()
        throttleOptimizer.step()
        train_loss += loss.item()
    for i, data in enumerate(throttleTestDataLoader):
        with torch.no_grad():
            images, labels = data
            predicted_output = throttleNet(images.float())
            loss = criterion(predicted_output, labels.long())
            test_loss += loss.item()
    train_loss = train_loss/len(throttleTrainDataLoader)
    test_loss = test_loss/len(throttleTestDataLoader)
    train_loss_history.append(train_loss)
    test_loss_history.append(test_loss)
    print('Epoch %s finished with train loss %s and test loss %s'%(epoch, train_loss, test_loss))

Epoch 0 finished with train loss 1964897.3804939035 and test loss 15594.791829427084
Epoch 1 finished with train loss 13683.445651049517 and test loss 165.5520807902018
Epoch 2 finished with train loss 86.07955437898636 and test loss 1.4783430099487305
Epoch 3 finished with train loss 0.7827622698886054 and test loss 0.37340808908144635
Epoch 4 finished with train loss 0.27322978085401106 and test loss 0.24841207390030226
Epoch 5 finished with train loss 0.28483382931777407 and test loss 0.24775553991397223
Epoch 6 finished with train loss 0.26653764412111164 and test loss 0.38949724783500034
Epoch 7 finished with train loss 0.26093929458637627 and test loss 0.24730967978636423
Epoch 8 finished with train loss 0.32216781134508093 and test loss 0.24813662841916084
Epoch 9 finished with train loss 0.32192883822990925 and test loss 0.32510771105686825


In [132]:
from torch import int64
import cv2 
import os 
import glob 
from google.colab.patches import cv2_imshow
import numpy as np
import torch
controlData = np.loadtxt('Robot Log/robot_log.csv', delimiter = ';', skiprows = 1, usecols = (1,2), dtype = float, max_rows = 1000)
controlData[:,0] /= np.max(controlData[:,0])
img_dir = "Images"
data_path = os.path.join(img_dir,'*g') 
files = glob.glob(data_path) 
Images = [] 
for f1 in files: 
    img = cv2.imread(f1, 0) 
    Images.append(img)
Images = np.array(Images)
Images = (np.divide(Images, 255))
Images = torch.tensor(Images)
TestDatasetSteer = TensorDataset(Images, torch.tensor(controlData))
TestDataloaderSteer = torch.utils.data.DataLoader(TestDatasetSteer, batch_size=1, shuffle=False)

for i, data in enumerate(TestDataloaderSteer):
  predicted_output_throttle = throttleNet(images.float())
  images, labels = data
  #print(labels)
  images = images.to(torch.float32)
  #print(np.shape(images))
  predicted_output_steer = steerNet(images.float())
  predicted_output_throttle = throttleNet(images.float())
  correct_steer = labels[0,0]
  correct_throttle = labels[0,1]
  print ("correct steer: " + str(correct_steer) + " predicted steer: " + str(predicted_output_steer))
  print ("correct throttle: " + str(correct_throttle) + " predicted throttle: " + str(predicted_output_throttle))

correct steer: tensor(0., dtype=torch.float64) predicted steer: tensor([[-1.4712]], grad_fn=<AddmmBackward0>)
correct throttle: tensor(0., dtype=torch.float64) predicted throttle: tensor([[0.2702]], grad_fn=<AddmmBackward0>)
correct steer: tensor(0., dtype=torch.float64) predicted steer: tensor([[-0.0742]], grad_fn=<AddmmBackward0>)
correct throttle: tensor(0., dtype=torch.float64) predicted throttle: tensor([[0.2702]], grad_fn=<AddmmBackward0>)
correct steer: tensor(0., dtype=torch.float64) predicted steer: tensor([[-2.6635]], grad_fn=<AddmmBackward0>)
correct throttle: tensor(0., dtype=torch.float64) predicted throttle: tensor([[0.2702]], grad_fn=<AddmmBackward0>)
correct steer: tensor(0., dtype=torch.float64) predicted steer: tensor([[-0.8356]], grad_fn=<AddmmBackward0>)
correct throttle: tensor(0., dtype=torch.float64) predicted throttle: tensor([[0.2702]], grad_fn=<AddmmBackward0>)
correct steer: tensor(0., dtype=torch.float64) predicted steer: tensor([[-0.0587]], grad_fn=<AddmmBa

KeyboardInterrupt: ignored